In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import undetected_chromedriver as uc
from time import sleep
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.common.by import By
from tqdm import tqdm

chargement de la page youtube + pas de détection comme étant rebot

In [6]:
if __name__ == "__main__":
    options = webdriver.ChromeOptions()
    options.add_argument('proxy-server=106.122.8.54:3128')
    options.add_argument('user-data-dir=other')
    browser = uc.Chrome(
        options=options,
    )
    browser.get("https://www.youtube.com")
    sleep(2)   
    browser.maximize_window()
    browser.execute_script("document.body.style.zoom='50%'")

la fonction scroll

In [2]:
def scroll():
    while(True):
        height = browser.execute_script("return document.body.scrollHeight")
        sleep(1)
        browser.find_element_by_tag_name('body').send_keys(Keys.END)
        if int(height)==0:
            break

In [21]:
with open ('pfa_links.txt','r') as f :
     videos_list = f.readlines()
len(videos_list)

162

In [22]:
# df_final= pd.read_csv('final.csv',index_col=0)
df_final = pd.DataFrame(columns=['author','comment'])
df_final

,author,comment


for each video in the list we have to repeat scrolling till the end to collect all comments

In [ ]:
for link in tqdm(videos_list) :
    browser.get(link)
    sleep(2)
    browser.execute_script('videos = document.querySelectorAll("video"); for(video of videos) {video.pause()}')
    title = browser.find_elements(by=By.XPATH, value='//h1/yt-formatted-string')[0].text
    youtuber = browser.find_element(by=By.XPATH, value='//div[@id="owner"]//div[@id="text-container"]//a').text

    last_position = browser.execute_script("return window.pageYOffset ;")
    while True:
        fin = True
        for i in range(3):
            scroll()
            sleep(2)
            curr_position = browser.execute_script("return window.pageYOffset ;")
            if (curr_position!=last_position):
                fin = False
                break
        if fin  :
            l = browser.find_elements(by=By.XPATH, value='//div[@id="contents"]/ytd-comment-thread-renderer//yt-formatted-string[@id="content-text"]')
            m = browser.find_elements(by=By.XPATH, value='//a[@id="author-text"]/span')
            authors = [n.text for n in m]
            comments = [p.text for p in l]
            break
        else: last_position = curr_position
    authors.insert(0,youtuber)
    comments.insert(0,title)
    
    df=pd.DataFrame({"author":authors , "comment":comments})
    df["author"] =df['author'].apply(lambda x: None if x == '' else x)
    df["comment"] =df['comment'].apply(lambda x: None if x == '' else x)

    df_final=pd.concat([df_final,df],axis=0).reset_index(drop=True)

In [25]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150573 entries, 0 to 150572
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   author   150081 non-null  object
 1   comment  149566 non-null  object
dtypes: object(2)
memory usage: 3.4+ MB


In [26]:
df_final.head()

,author,comment
0,Dr. Eric Berg DC,How Your Feet Are Warning You About Your Liver...
1,Rukhsana Begum,"Someone actually teaching us about our health,..."
2,Adel Qutob,3 months ago I watched the episode about intim...
3,Liz Zy,I wished every med doctor would spread this kn...
4,Galavanter,I was a truck driver 40lbs over weight.\nFollo...


In [27]:
df_final.tail()

,author,comment
150568,John Amato,Adaptation. Not evolution. Evolution isnt a re...
150569,She’s Niki,It’s sad when you talk on the negative effects...
150570,Bossco,this guy look like he has been fasting for a ...
150571,kricku,tl;dr\nDead
150572,M TABARIK ASIF,I'm muslim and proud to say we fast for one mo...


In [16]:
df_final.to_csv("final.csv")